## <font color=darkcyan> Variational inference </font>
#### <font color=darkorange> Basics: Evidence Lower Bound (ELBO) & Coordinate Ascent Variational Inference (CAVI) 

In [ ]:
"""""""""""""""""
Required packages
"""""""""""""""""
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

On considère tout d'abord le cas d'étude de l'article : ``Variational Inference: A Review for Statisticians, Blei et al; (2017)``, un mélange de gaussiennes de variance 1. 

#### Mélange de gaussiennes

On considère un mélange de $K$ gaussiennes de moyennes $\mu = (\mu_k)_{1\leqslant k \leqslant K}$ et de variance 1. Les variables $\mu = (\mu_k)_{1\leqslant k \leqslant K}$ sont (i.i.d.) de loi  gaussienne de moyenne 0 et de variance $\sigma^2$. Le poids de la composante $k$ est noté $\omega_k$. Conditionnellement à $\mu$, les observations $(X_i)_{1\leqslant i\leqslant n}$ sont i.i.d. et la densité de probabilité de $X_1$ est:

$$
p(x|\mu) = \sum_{k=1}^K \omega_k \varphi_{\mu_k,1}(x)\,,
$$

où $\varphi_{\mu_k,\sigma^2}$ est la densité gaussienne de moyenne $\mu_k$ et de variance $\sigma^2$. La vraisemblance jointe est alors :

$$
p(x_1,\cdots,x_n) = \int p(x_1,\cdots,x_n|\mu) p(\mu) \mathrm{d} \mu = \int \prod_{i=1}^n p(x_i|\mu) p(\mu) \mathrm{d} \mu = \int \prod_{i=1}^n \left(\sum_{k=1}^K \omega_k \varphi_{\mu_k,1}(x_i)\right) p(\mu) \mathrm{d} \mu
$$

#### Question 1
Ecrire une fonction qui simule un échantillon de cette loi avec $K= 3$, $\sigma^2 = 5$, $\omega_k = 1/K$ pour tout $1\leqslant k \leqslant K$.

In [ ]:
# Sample data
K  = 3 # number of mixture components
mu = np.random.normal(0,np.sqrt(5),3) # means of the distribution in each cluster
n_samples = 1000 # number of samples

Notre objectif est d'approcher $p(\mu,c|x)$ où $c = (c_1,\cdots,c_n)$ sont les composantes des observations.  L'approximation `mean-field` considérée s'écrit:

$$
q(\mu,c) = \prod_{k=1}^K \varphi_{m_k,s_k}(\mu_k)\prod_{i=1}^n \mathrm{Cat}_{\phi_i}(c_i)\,, 
$$

ce qui signifie que:

- $\mu$ et $c$ sont indépendantes.
- $(\mu_{k})_{1\leqslant k \leqslant K}$ sont des gaussiennes indépendantes de moyennes $(m_{k})_{1\leqslant k \leqslant K}$ et variances $(s_{k})_{1\leqslant k \leqslant K}$.
- $(c_{i})_{1\leqslant i \leqslant n}$ sont indépendantes de distribution multinomiales de paramètres $(\phi_i)_{1\leqslant i \leqslant n}$: $q(c_i=k) = \phi_i(k)$ pour $1\leqslant k \leqslant K$. 

Notons $\mathcal{D}$ la famille des distributions où les moyennes $(m_{k})_{1\leqslant k \leqslant K}\in \mathbb{R}^K$, les variances $(s_{k})_{1\leqslant k \leqslant K}\in (\mathbb{R}_+^*)^K$ et les  $(\phi_i)_{1\leqslant i \leqslant n}\in \mathcal{S}_K^n$ où $\mathcal{S}_K$ est le simplexe de dimension $K$. 

L'objectif est de trouver le
"meilleur candidat" dans $\mathcal{D}$ pour approcher $p(\mu,c|x)$, i.e. celui qui minimise ``la distance de Kullback suivante``:

$$
q^* = \mathrm{Argmin}_{q\in\mathcal{D}} \mathrm{KL}\left(q(\mu,c)\|p(\mu,c|x)\right)\,.
$$

Notez que :
\begin{align*}
\mathrm{KL}\left(q(\mu,c)\|p(\mu,c|x)\right) &= \mathbb{E}_q[\log q(\mu,c)] - \mathbb{E}_q[\log p(\mu,c|x)]\,,\\
 &= \mathbb{E}_q[\log q(\mu,c)] - \mathbb{E}_q[\log p(\mu,c,x)]+\log p(x)\,,\\
&= -\mathrm{ELBO}(q)+\log p(x)\,,
\end{align*}

où l'``Evidence Lower Bound`` (ELBO) est

$$
\mathrm{ELBO}(q) = -\mathbb{E}_q[\log q(\mu,c)] + \mathbb{E}_q[\log p(\mu,c,x)] \,.
$$

Ainsi, ``minimiser la divergence de Kullback`` revient à maximiser la ELBO, avec $\log p(x)\geqslant \mathrm{ELBO}(q)$.

La complexité de $\mathcal{D}$ détermine la complexité du problème d'optimisation.

#### Generic CAVI algorithm

L'algorithme CAVI calcule itérativement pour $1\leqslant k \leqslant K$,

$$
q(\mu_k) \propto \mathrm{exp}\left(\mathbb{E}_{\tilde q_{\mu_k}}[\log \tilde p_k(\mu_k|x)]\right)
$$

et pour tout  $1\leqslant i \leqslant n$,

$$
q(c_i) \propto \mathrm{exp}\left(\mathbb{E}_{\tilde q_{c_i}}[\log \tilde p_i(c_i|x)]\right)\,,
$$

où

- $\tilde p_i(c_i|x)$ est la distribution conditionnelle de $c_i$ sachant les observations et les autres paramètres et $\tilde p_k(\mu_k|x)$ est la loi conditionnelle de$\mu_k$ sachant les observations et les autres paramètres.

- $\mathbb{E}_{\tilde q_z}$ est l'espérance sous la loi variationnelle de toutes les variables sauf $z$.

#### <font color=darkorange> Application au mélange de lois gaussiennes </font>

#### Question 2
Ecrire la mise à jour explicitement pour les $c_i$, $1\leqslant i \leqslant n$.

#### Mise à jour de  $(\phi_i)_{1\leqslant i \leqslant n}$ avec CAVI

#### Question 3
Ecrire une fonction effectuant la mise à jour des $c_i$, $1\leqslant i \leqslant n$, les autres paramètres étant fixés.

In [ ]:
def CAVI_update_phi(X,m,s2):
    """
    Inputs
    ----------
    X: data
    m: current estimation of the m_k
    s2: current estimation of the s_k
    
    Outputs
    -------
    phi: new estimation of phi
    """
    
    # A compléter

#### Question 4
Ecrire la mise à jour explicitement pour les $(m_{k})_{1\leqslant k \leqslant K}$ et des $(s_{k})_{1\leqslant k \leqslant K}$.

#### Update of $(m_{k})_{1\leqslant k \leqslant K}$ and $(s_{k})_{1\leqslant k \leqslant K}$ using CAVI

#### Question 5
- Ecrire une fonction effectuant la mise à jour des $(m_{k})_{1\leqslant k \leqslant K}$ et des $(s_{k})_{1\leqslant k \leqslant K}$, les autres paramètres étant fixés.
- Ecrire une fonction calculant la ELBO.

In [ ]:
def CAVI_update_mu_s2(X,m,phi,s2,sigma2):
    """
    Inputs
    ----------
    X: data
    m: current estimation of the m_k
    s2: current estimation of the s_k
    phi: current estimation of phi
    sigma2: current estimation of sigma^2
    
    Outputs
    -------
    m, s2: new estimation of the m_k and the s_k
    """
    # A compléter

In [ ]:

def elbo(X,phi,m,s2,sigma2):
    """
    Inputs
    ----------
    X: data
    m: current estimation of the m_k
    s2: current estimation of the s_k
    phi: current estimation of phi
    sigma2: current estimation of sigma^2
    
    Outputs
    -------
    elbo: value of the elbo with the input parameters
    """
    
    # A compléter

#### Question 6
- Ecrire une fonction effectuant les mises à jour itératives de l'algorithme CAVI.
- Mettre en oeuvre l'algoroithme et donner la ELBO et les estimations au fil des itérations.

In [ ]:
def CAVI_mixture_Gaussian(X,m, s2, phi, sigma2, max_iter = 500, epsilon = 1e-8):
    """
    Inputs
    ----------
    X: data
    m: initial estimation of the m_k
    s2: initial estimation of the s_k
    phi: initial estimation of phi
    sigma2: initial estimation of sigma^2
    
    Outputs
    -------
    elbos: value of the elbo long iterations
    m_est, s2_est: sequence of estimators along iterations
    """
    
    # A compléter

#### Run the algorithm

#### Sensitibilité aux conditions initiales

#### Question 7
Etudiez la sensibilité aux conditions initiales